In [6]:
# Import
import os
import json
import shutil
import cv2

### Installation von YOLOv5 mit Git und pip

In [ ]:
!git clone "https://github.com/ultralytics/yolov5.git"
%cd yolov5
%pip install -qr requirements.txt
%cd ..

### Kopieren von Bildern in den YOLOv5-Dataset-Ordner und Konvertieren von JSON-Daten in das YOLOv5-Format (txt)

YOLOv5 txt Format:  
0 0.4855699855699856 0.3187725631768953 0.3203463203463203 0.3689530685920578  
Koordinaten werden Normalisiert, Höhe und Breite d. Bounding Box (in Abhängigkeit von der Bildgröße). Ersetzen der Klassennamen durch Klassenindizes.

In [7]:
# Bilder in yolov5/dataset/images kopieren
def copy_images_to_dataset(images_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(images_dir):
        image_path = os.path.join(images_dir, filename)
        output_path = os.path.join(output_dir, filename)

        shutil.copy(image_path, output_path)

In [11]:
# JSON Daten in YOLOv5-Format konvertieren. Normalisieren der Koordinaten, Höhe und Breite d. Bounding Box 
# (in Abhängigkeit von der Bildgröße). Ersetzen der Klassennamen durch Klassenindizes. Erstellung einer
# classes.txt-Datei mit den Klassennamen und den zugehörigen Klassenindizes.

def convert_to_yolov5_format(json_dir, image_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    class_mapping = {}
    class_index = 0

    # Erstelle YOLOv5-Annotationen
    for filename in os.listdir(json_dir):
        if filename.endswith(".json"):
            json_path = os.path.join(json_dir, filename)
            image_filename = os.path.splitext(filename)[0]
            image_path = os.path.join(image_dir, image_filename)

            with open(json_path, 'r') as f:
                json_data = json.load(f)

            image = cv2.imread(image_path)
            image_height, image_width, _ = image.shape

            yolo_annotations = []
            for bbox in json_data["Annotations"]:
                x_min, y_min, x_max, y_max = bbox["BoundingBox"]
                class_name = bbox["classname"]

                if class_name not in class_mapping:
                    class_mapping[class_name] = class_index
                    class_index += 1

                class_index = class_mapping[class_name]

                # Normalisierung der Koordinaten
                x_center = (x_min + x_max) / (2 * image_width)
                y_center = (y_min + y_max) / (2 * image_height)
                width = (x_max - x_min) / image_width
                height = (y_max - y_min) / image_height

                yolo_annotation = f"{class_index} {x_center} {y_center} {width} {height}"
                yolo_annotations.append(yolo_annotation)

            output_filename = f"{image_filename.split('.')[0]}.txt"
            output_path = os.path.join(output_dir, output_filename)

            with open(output_path, 'w') as f:
                for annotation in yolo_annotations:
                    f.write(annotation + '\n')

    # Erstelle classes.txt-Datei
    classes_file_path = os.path.join(output_dir, "classes.txt")
    with open(classes_file_path, 'w') as f:
        for class_name, class_index in class_mapping.items():
            f.write(f"{class_name}\n")

In [12]:
# Pfad zu den JSON-Daten und Bildern
json_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations" 
image_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images"  

# Pfad zu den YOLOv5 Dataset-Ordnern
labels_output_dir = "yolov5/dataset/labels"  
image_output_dir = "yolov5/dataset/images"  

In [13]:
# Konvertierung
if os.path.exists(image_dir) and os.path.exists(json_dir):
    copy_images_to_dataset(image_dir, image_output_dir)
    convert_to_yolov5_format(json_dir, image_dir, labels_output_dir)
    print("Konvertierung erfolgreich.")
else:
    if not os.path.exists(image_dir):
        print("Image Pfad existiert nicht.")
    elif not os.path.exists(json_dir):
        print("Annotations Pfad existiert nicht.")
    else:
        print("unbekannter Fehler.")

Konvertierung erfolgreich.


### Training von YOLOv5 mit WandB-Integration und Upload des Datasets

conda install pytorch=1.9.0 cudatoolkit=11.1 -c pytorch -c conda-forge  
damit es läuft
torchvision               0.10.0+cu111

In [15]:
import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3080', major=8, minor=6, total_memory=10239MB, multi_processor_count=68)


In [16]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saniye-ogul98 (chillibuddies). Use `wandb login --relogin` to force relogin


True

In [17]:
%cd yolov5

c:\Users\Saniye\Documents\GitHub\Deep-Vision-sta\YOLO\yolov5


In [18]:
!python train.py --data dataset/data.yaml --epochs 10 --weights '' --cfg yolov5s.yaml  --batch-size 4 --upload_dataset --bbox_interval 1 --img 640

wandb: WARNING  wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: saniye-ogul98 (chillibuddies). Use `wandb login --relogin` to force relogin
train: weights='', cfg=yolov5s.yaml, data=dataset/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=10, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=True, bbox_interval=1, artifact_alias=latest
github:  YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
Traceback (most r